In [ ]:
%pip install --user --quiet ciw
import ciw

## Simulation

In [ ]:
from dataclasses import dataclass, asdict

@dataclass
class Model:
    num_servers: int
    queue_capacity: int
    arrival_dist: ciw.dists.Distribution
    service_dist: ciw.dists.Distribution

@dataclass
class SimResult:
    model: Model
    task_count: int
    utilization: float
    loss_probability: float
    mean_wait_time: float
    mean_residence_time: float
    
    def __init__(self, model: Model, sim: ciw.Simulation):
        self.model = model
        self.utilization = sim.transitive_nodes[0].server_utilisation
        
        tasks = sim.get_all_records()
        self.task_count = len(tasks)
        self.loss_probability = len(sim.rejection_dict[1][0]) / self.task_count
        self.mean_wait_time = sum(t.waiting_time for t in tasks) / self.task_count
        self.mean_residence_time = sum(t.waiting_time + t.service_time for t in tasks) / self.task_count

def simulate(model: Model, task_count: int) -> SimResult:
    sim = ciw.Simulation(ciw.create_network(
        arrival_distributions=[model.arrival_dist],
        service_distributions=[model.service_dist],
        number_of_servers=[model.num_servers],
        queue_capacities=[model.queue_capacity]
    ))
    sim.simulate_until_max_customers(task_count, method='Finish')
    return SimResult(model, sim)

## Distributions

In [ ]:
def dexp(rate):
    return ciw.dists.Exponential(rate)

def dhypoexp2(rate1, rate2):
    return dexp(rate1) + dexp(rate2)

def dtrace():
    with open('trace.txt', 'r') as f:
        trace = [float(v) for v in f.read().splitlines()]
        return ciw.dists.Sequential(trace)

## Model Variations

In [ ]:
models = [
    Model(num_servers=2, queue_capacity=10, arrival_dist=dexp(1 / 20), service_dist=dexp(1 / 18)),
    Model(num_servers=2, queue_capacity=10, arrival_dist=dtrace(), service_dist=dexp(1 / 18)),
    Model(num_servers=2, queue_capacity=10, arrival_dist=dhypoexp2(1 / 8.55, 1 / 1.3), service_dist=dexp(1 / 18))    
]

## Analysis

In [ ]:
import pandas as pd

results = [simulate(m, 1000) for m in models]
pd.io.json.json_normalize(map(asdict, results)).drop(['model.arrival_dist', 'model.service_dist'], 1)